In [18]:
import mlflow
import os
import getpass
import subprocess
import time

from domino import Domino

from langchain import LLMMathChain
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool, DuckDuckGoSearchRun

from pydantic import BaseModel, Field

In [8]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key: ········


In [9]:
llm = ChatOpenAI(temperature=0)

In [35]:
search = DuckDuckGoSearchRun()

search_tool = Tool.from_function(
    func=search.run,
    name="Search",
    description="useful for when you need to search the internet for information"
)

llm_math_chain = LLMMathChain(llm=llm, verbose=True)

math_tool = Tool.from_function(
    func=llm_math_chain.run,
    name="Calculator",
    description="Useful for when you are asked to perform math calculations"
)

# create an experiment in MLflow
@tool("create_experiment", return_direct=True)
def create_experiment(experiment_name:str):
    """Create an experiment in Domino"""
    # we'll make the name unique in the project by appending a timestamp so that you and other users can run this cell more than once.
    timestamp = time.time()
    username = os.environ['DOMINO_STARTING_USERNAME']
    if experiment_name:
        experiment_name = f"{experiment_name}-{timestamp}"
        # below, we'll use the returned experiment_id in calls to mlflow.start_run() to add data to the experiment.
        experiment_id = mlflow.create_experiment(experiment_name.replace("experiment_name = \"", ""))
        print(f"Experiment id: {experiment_id}")
        print(f"Experiment name: {experiment_name}")


# execute a blocking job in Domino
@tool("create_run_job", return_direct=True)
def create_run_job(job_file:str):
    """Create and run  a job in Domino"""
    
    domino = Domino(
    f"{owner}/{project}",
    api_key=os.environ["DOMINO_USER_API_KEY"],
    host=os.environ["DOMINO_API_HOST"],
    )
    if job_file:
        # Blocking: this will start the run and wait for the run to finish before returning the status of the run
        domino_run = domino.runs_start_blocking(
            [job_file], title="Started from Domino agent"
        )
    print(domino_run)


/home/ubuntu/.local/lib/python3.9/site-packages/langchain/chains/llm_math/base.py:56: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [36]:
tools = [search_tool, math_tool,create_experiment,create_run_job]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [14]:
agent.run("Search for the Sacramento Kings win-loss record for the entire 2022-2023 season. Add games won and games lost and multiply that result by 2.")



> Entering new AgentExecutor chain...
I need to search for the Sacramento Kings win-loss record for the 2022-2023 season. Then I need to add the games won and games lost and multiply the result by 2.
Action: Search
Action Input: "Sacramento Kings win-loss record 2022-2023 season"
Observation: Stay Up-To-Date With The Latest Sacramento Kings Schedule, Live Scores, And Results For The 2022-2023 NBA Season! ... 2022-2023 Betting Record. Win / Loss. 48-34. Against the Spread. And when the final buzzer sounded on the regular season, with 9,846 points, the team set a new single-season franchise record for points scored, surpassing the 1961-62 Cincinnati Royals by one point. Sacramento finishes Q1 at DEN with 43 points and has now scored 9,846 points this season. Stats Roster Depth Chart Transactions Injuries StubHub Next Game Fri, Jan 5 @ 10:00 pm ET vs Toronto Raptors (14-20) Golden 1 Center SAC -5, O/U 238.5 Game Preview Watch on regional restrictions... The Sacramento Kings put together

'82'

In [37]:
agent.run("create an experiment called agent_exp")



> Entering new AgentExecutor chain...
I need to create an experiment called "agent_exp".
Action: create_experiment
Action Input: experiment_name = "agent_exp"Experiment id: 8
Experiment name: experiment_name = "agent_exp-1704828129.942171

Observation: None


> Finished chain.


In [ ]:
agent.run("run a job using job.py")